In [ ]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [ ]:
!pip install keras-image-helper

In [4]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [17]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor

In [8]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [7]:
url = 'http://bit.ly/mlbookcamp-pants'

In [11]:
X = preprocessor.from_url(url)

In [12]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [ ]:
np_to_protobuf(X)

In [15]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [21]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [22]:
pb_response 

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8798640966415405
    float_val: -4.756310939788818
    float_val: -2.359532594680786
    float_val: -1.0892654657363892
    float_val: 9.903782844543457
    float_val: -2.826178789138794
    float_val: -3.6483113765716553
    float_val: 3.2411556243896484
    float_val: -2.612095594406128
    float_val: -4.8520355224609375
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [27]:
preds = pb_response.outputs['dense_7'].float_val

In [25]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [29]:

dict(zip(classes, preds))

{'dress': -1.8798640966415405,
 'hat': -4.756310939788818,
 'longsleeve': -2.359532594680786,
 'outwear': -1.0892654657363892,
 'pants': 9.903782844543457,
 'shirt': -2.826178789138794,
 'shoes': -3.6483113765716553,
 'shorts': 3.2411556243896484,
 'skirt': -2.612095594406128,
 't-shirt': -4.8520355224609375}